# Importing the Libraries

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential

# Loading the MNIST Dataset

In [ ]:
# loading the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Visualizing the Data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
fig = plt.figure()

# view some digits visually
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.tight_layout()
    plt.imshow(X_train[i], interpolation='none')
    plt.title("Digit: {}".format(y_train[i]))

    # don't show the x and y-ticks
    plt.xticks([])
    plt.yticks([])


In [ ]:
print(X_train[0])
print(y_train[0])


# Getting the Images Dimension

In [ ]:
from tensorflow.keras import backend as k
from tensorflow.keras import layers

print(X_train.shape)

# image dimensions
img_width, img_height = X_train.shape[1], X_train.shape[2]


# Reshaping and Normalizing the Data

In [ ]:
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

input_shape = (img_width, img_height, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

print('X_train shape:', X_train.shape) 


# One-Hot Encoding

In [ ]:
# set number of categories
num_category = 10

# convert class vectors to binary class matrices - one-hot encoding
y_train = keras.utils.to_categorical(y_train, num_category)
y_test = keras.utils.to_categorical(y_test, num_category)


# Building the Model

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten
from tensorflow.keras.layers import Dropout

model = Sequential()


In [ ]:
#**********************
# Convolutional Layer 1
#**********************
# 32 convolution filters each of size 3x3
model.add(Conv2D(32,                      # number of filters
                 kernel_size=(3, 3),      # filter size
                 activation='relu',       # activation function   
                 strides=1,               # stride 
                 padding="same",          # padding; ensures the output 
                                          # will be the same size (28x28)
                 input_shape=input_shape))

#**********************
# Convolutional Layer 2
#**********************
# 64 convolution filters each of size 3x3
model.add(Conv2D(64,                      # number of filters 
                 (3, 3),                  # filter size                  
                 activation='relu'))      # no padding, hence output 
                                          # size will be reduced (26x26)

#***************************
# Max Pooling, pool size 2x2
#***************************
# choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2))) # output size will be halved (13x13)
                                          # default stride is the size of the
                                          # pool

#********
# Dropout
#********
# randomly turn neurons on and off to improve convergence
model.add(Dropout(0.25))

#**********************
# Flatten
#**********************
# flatten since too many dimensions, we only want a classification output
model.add(Flatten())

#**********************
# Dense
#**********************
# fully connected to get all relevant data
model.add(Dense(128, activation='relu'))

#**********************
# Dropout
#**********************
model.add(Dropout(0.5))

#**********************
# Dense
#**********************
# output a softmax to squash the matrix into output probabilities
model.add(Dense(num_category, activation='softmax'))


# Viewing the Summary of the Model

In [ ]:
model.summary()


In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='cnnmodel.png', show_shapes=True, show_layer_names=True)


# Compiling the Model

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])


# Training the Model

In [ ]:
batch_size = 64
num_epoch = 15
VALIDATION_SPLIT = 0.2               # how much TRAIN is reserved for VALIDATION

# model training
model_log = model.fit(X_train, y_train,
                      batch_size=batch_size,
                      epochs=num_epoch,
                      verbose=1,
                      validation_split=VALIDATION_SPLIT)


# Evaluating the Model

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])


# Plotting the Accuracy and Loss

In [ ]:
import os

fig = plt.figure()
plt.subplot(1,2,1)
plt.plot(model_log.history['accuracy'])
plt.plot(model_log.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(1,2,2)
plt.plot(model_log.history['loss'])
plt.plot(model_log.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.tight_layout()


# Saving the Model

In [ ]:
from tensorflow.keras.models import load_model

model.save('trained_cnn_model.h5')  # creates a HDF5 file 'trained_model.h5'
del model  # deletes the existing model


# Loading the Model

In [ ]:
# returns a compiled model identical to the previous one
model = load_model('trained_cnn_model.h5')


# Performing Predictions

In [ ]:
import numpy as np

# the item to predict
#========
index = 6
#========

# shape of X_test
print(X_test.shape)                  # (10000, 28, 28, 1)

# shape of the first item in X_test  # (28, 28, 1)
print(X_test[index].shape)

x = X_test[index].reshape(1,28,28,1)
print(x.shape)                       # (1, 28, 28, 1)

# you can do the prediction now
print(model.predict(x))

print(y_test[index])


In [ ]:
# print(model.predict_classes(x)) # deprecated     # [4]

print(np.argmax(model.predict(x), axis=-1))

# see the original value
print(y_test[index])


# Converting the Model to TensorFlow Lite

In [ ]:
# tensorflow 1.x
import tensorflow.compat.v1 as tf

# load the trained model
converter = tf.lite.TFLiteConverter.from_keras_model_file('trained_cnn_model.h5')

# convert it to tensorflow lite
tflite_model = converter.convert()

# write the tensorflow lite model to disk
open("trained_cnn_model.tflite", "wb").write(tflite_model)
